In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import load_data, save_data
from utilities import temporal_train_test_split
from utilities import print_title, print_label

In [2]:
# Data manipulation and analysis
import pandas as pd

# Date and time manipulation
from datetime import date

# File and directory manipulation
from pathlib import Path

# Data preprocessing and model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Model evaluation metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

# Machine learning models (regression)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# Ensemble methods
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
def split_dataset_by_date(raw_data: pd.DataFrame, todays_date: str):
    # Filter data by today's date
    filter_data_by_date = raw_data["Date"] == todays_date
    
    # Create a new dataframe with today's data
    todays_data = raw_data[filter_data_by_date].reset_index(drop=True)
    
    # Create a new dataframe with historical data (excluding today's data)
    historical_data = raw_data[~filter_data_by_date].reset_index(drop=True)
    
    return historical_data, todays_data

In [4]:
file_name = "sp500_adj_close_raw_with_nas"
file_path = f"../../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║File `test_w_na.csv.bz2` loaded from `sp500_adj_close_raw_with_nas.zip`║
╚═══════════════════════════════════════════════════════════════╝


### Data Pre-Processing
___

#### Split todays data (For prediction) and historical data (For training)

In [5]:
todays_date = "2024-10-25"

historical_data, todays_data = split_dataset_by_date(raw_data, todays_date)

print("Todays Date:", todays_date)

Todays Date: 2024-10-25


#### Ensure there are missing values on Todays Data is what we are predicting...

In [6]:
todays_data.isnull().sum()

Date                    0
Ticker                  0
Adjusted Close          0
Next Day Close        501
Previous Day Close      0
Return                  0
Volatility              0
RSI                     0
SMA_50                  0
SMA_100                 0
SMA_200                 0
Upper Band              0
Lower Band              0
Support                 0
Resistance              0
Action                501
dtype: int64

#### Handle missing values (NA's) on the historical data used to train and test...

In [7]:
def clean_historical_data(historical_data: pd.DataFrame) -> pd.DataFrame:
    # Remove all rows where `Adjusted Close` is 0
    historical_data = historical_data[historical_data["Adjusted Close"] != 0].reset_index(drop=True)
    
    # Remove all rows where `Volatility` is NaN
    historical_data = historical_data.dropna(subset=["Volatility"]).reset_index(drop=True)
    
    # Backfill the `RSI` column
    historical_data["RSI"] = historical_data["RSI"].bfill()
    
    # Backfill the `Action` column
    historical_data["Action"] = historical_data["Action"].bfill()
    
    return historical_data

historical_data = clean_historical_data(historical_data)

historical_data.isnull().sum()

Date                  0
Ticker                0
Adjusted Close        0
Next Day Close        0
Previous Day Close    0
Return                0
Volatility            0
RSI                   0
SMA_50                0
SMA_100               0
SMA_200               0
Upper Band            0
Lower Band            0
Support               0
Resistance            0
Action                0
dtype: int64

### Exploratory Data Analysis (EDA):
___

#### Todays Data:

In [8]:
print("Today's Data Shape:", todays_data.shape)

Today's Data Shape: (501, 16)


#### Historical Data:

In [9]:
print("Historical Data Shape:", historical_data.shape)

Historical Data Shape: (1978479, 16)


### Create Multiple Versions of Dataset
___

#### Select which version of the data to work with

In [10]:
main_data = historical_data.copy().reset_index(drop=True)

# Create multiple versions of the dataset

# Data with out dates and tickers (Set as index for reference)
data_v1 = main_data.copy().set_index(["Date", "Ticker"])

# Data with dates and without tickers (Set as index for reference)
def prepare_data_v2(main_data: pd.DataFrame) -> pd.DataFrame:
    # Create a copy of the input DataFrame
    df = main_data.copy().reset_index(drop=True)
    
    # Convert the `Date` column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from the `Date` column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Set the index to `Date` and `Ticker`
    df = df.set_index(["Date", "Ticker"])
    
    return df

data_v2 = prepare_data_v2(main_data)

##### **Note: It is recommended to remove `["Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]` after VIF inspection...**

##### **Note: It is recommended to remove `["Day"]` after p-value inspection...**

In [11]:
select_data = data_v2.copy()

select_columns_to_drop = ["Action", "Previous Day Close", "SMA_50", "Resistance", "Upper Band", "SMA_200", "Day"]

data = select_data.drop(columns=select_columns_to_drop)

print("Shape:", data.shape)
data.head()

Shape: (1978479, 10)


Adjusted Close  Next Day Close    Return  Volatility  \
Date       Ticker                                                         
2008-01-02 A            23.256388       23.025743 -0.011976    0.015704   
           AAPL          5.876342        5.879055 -0.016357    0.018937   
           ABT          18.130209       18.019754 -0.006055    0.010484   
           ACGL          7.608889        7.764444 -0.026581    0.016022   
           ACN          26.437077       25.982521  0.000833    0.024039   

                         RSI    SMA_100  Lower Band    Support  Year  Month  
Date       Ticker                                                            
2008-01-02 A       48.827590  23.299887   22.540236  21.392029  2008      1  
           AAPL    59.067350   4.939064    5.403559   4.637376  2008      1  
           ABT     34.677372  17.628250   18.221804  16.775562  2008      1  
           ACGL    45.154190   7.878933    7.378535   7.463333  2008      1  
           ACN     54.812176  27.784420   24.273775  24.765512  2008      1

### Split data features `X` and target `y`
___

In [12]:
# Split the data into features (X) and target (y)
X = data.drop(columns="Next Day Close")

y = data["Next Day Close"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (1978479, 9)
Shape of y: (1978479,)


### Data Splitting
___

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, # 80% training and 20% testing
    random_state=42
)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1582783, 9)
Shape of X_test: (395696, 9)
Shape of y_train: (1582783,)
Shape of y_test: (395696,)


### Scaling the Data
___

#### Scale the data using `StandardScaler`

In [14]:
# Scale using StandardScaler
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)
print("y_test_scaled shape:", y_test_scaled.shape)

X_train_scaled shape: (1582783, 9)
X_test_scaled shape: (395696, 9)
y_train_scaled shape: (1582783, 1)
y_test_scaled shape: (395696, 1)


### Model Training
___


In [15]:
print(

SyntaxError: incomplete input (149104261.py, line 1)

In [ ]:
knn_regressor = KNeighborsRegressor()

param_grid_knn = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]
}

grid_search_knn = GridSearchCV(
    knn_regressor,
    param_grid_knn,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search_knn.fit(X_train_scaled, y_train_scaled)

In [ ]:
import json

print(json.dumps(grid_search_xgb.best_params_, indent=4))

### Model Evaluation
___

In [22]:
def adj_r2_score(model, X, y):
    r2 = model.score(X, y)
    n_cols = X.shape[1]
    return 1 - (1 - r2) * (len(y) - 1) / (len(y) - n_cols - 1)

In [23]:
best_xgb_model = grid_search_xgb.best_estimator_

In [ ]:
y_predict_scaled = best_xgb_model.predict(X_test_scaled)

y_predict_unscaled = y_scaler.inverse_transform(y_predict_scaled.reshape(-1, 1))
y_test_unscaled = y_scaler.inverse_transform(y_test_scaled.reshape(-1, 1))

mse = mean_squared_error(y_test_unscaled, y_predict_unscaled)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_unscaled, y_predict_unscaled)
adj_r2 = adj_r2_score(best_xgb_model, X_test_scaled, y_test_scaled)

print_title("XGBoost Regressor", closed_corners=False)
print_label("Mean Squared Error:", mse)
print_label("Root Mean Squared Error:", rmse)
print_label("R-Squared (Training Data):", best_xgb_model.score(X_train_scaled, y_train_scaled))
print_label("R-Squared (Testing Data):", r2)
print_label("Adjusted R-Squared:", adj_r2, closed_corners=True) 

In [ ]:
cv_scores = cross_val_score(
    best_xgb_model,
    X_train_scaled,
    y_train_scaled,
    cv=5,
    scoring="r2"
)

print_title("Cross Validation Scores", closed_corners=False)

for index, score in enumerate(cv_scores):
    print_label(f"Fold {index+1}:", score)

print_label("", "")
print_label("Mean R^2 Score:", cv_scores.mean())
print_label("Standard Deviation:", cv_scores.std(), closed_corners=True)

### Predict tomorrows `Adjusted Close`
___

In [ ]:
def preprocess_todays_data(df, columns_to_drop):
    # Convert 'Date' column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from 'Date' column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Drop specified columns
    df = df.drop(columns=columns_to_drop)
    
    # Set 'Date' and 'Ticker' as the index
    df = df.set_index(["Date", "Ticker"])
    
    # Drop 'Next Day Close' column
    df = df.drop(columns="Next Day Close")
    
    return df

X_to_predict = todays_data.copy()

X_to_predict = preprocess_todays_data(X_to_predict, select_columns_to_drop)

X_to_predict_scaled = X_scaler.transform(X_to_predict)

print("Shape of todays_data_clean_scaled:", X_to_predict_scaled.shape)
X_to_predict_scaled[:5]

In [ ]:
y_to_predict_scaled = best_xgb_model.predict(X_to_predict_scaled)

y_to_predict_unscaled = y_scaler.inverse_transform(y_to_predict_scaled.reshape(-1, 1))

print("Shape of y_to_predict_unscaled:", y_to_predict_unscaled.shape)
y_to_predict_unscaled[:5]

In [ ]:
prediction_df = todays_data.copy()

prediction_df["Next Day Close"] = y_to_predict_unscaled.round(3)

prediction_df = prediction_df[["Date", "Ticker", "Adjusted Close", "Next Day Close"]]

prediction_df = prediction_df.set_index(["Date", "Ticker"])

print("Shape:", prediction_df.shape)
display(prediction_df.head(20))
display(prediction_df.tail(20))

In [ ]:
# Make index column a regular column
prediction_df.reset_index(inplace=True)

prediction_df

In [ ]:
file_name = "XGB_predict.zip"
file_path = f"../../../data/raw_data/{file_name}"

save_data(prediction_df, file_path)